In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
import os

#######Data set preprocessing ########
 # Set dataset directory and parameters
dataset_dir = "C:/Users/NAMITHAA/Downloads/eee_dataset/AID"
target_size = (224, 224)  # Target size for DenseNet121
batch_size = 32

# Define the data generators with increased augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=45,  # Increased rotation range
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of the data for validation
)

# Data generators for training and validation
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Set as training data
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

####### Model Building: #######

   # Load DenseNet121 with pre-trained ImageNet weights
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of DenseNet121
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # Reduced the number of units
x = Dropout(0.3)(x)  # Reduced dropout rate
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)#defining the input and output layers.​

######## Model Training #########

# Fine-tune more layers of DenseNet121
for layer in base_model.layers[:-50]:
    layer.trainable = False #freezing top layers 
for layer in base_model.layers[-50:]:
    layer.trainable = True #proceed with  second part 

# Compile the model with a lower learning rate and learning rate scheduler
optimizer = Adam(learning_rate=0.0001)  # Lower learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
filepath = "C:/Users/NAMITHAA/Downloads/save_models.keras"

#When the monitored metric improves, it saves the model weights
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# adjusts the learning rate during training based on changes in the validation loss, aiming to improve model performance
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.000001, verbose=1)

#stops training if the validation loss does not improve for 15 consecutive epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model with more epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,  
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=1
)

#####Evalvation####
 # Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

Found 8000 images belonging to 30 classes.
Found 2000 images belonging to 30 classes.
Epoch 1/10


C:\Users\NAMITHAA\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2653 - loss: 2.8073
Epoch 1: val_accuracy improved from -inf to 0.77520, saving model to C:/Users/NAMITHAA/Downloads/save_models.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 717s 3s/step - accuracy: 0.2661 - loss: 2.8043 - val_accuracy: 0.7752 - val_loss: 0.9972 - learning_rate: 1.0000e-04
Epoch 2/10


c:\Program Files\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)



Epoch 2: val_accuracy did not improve from 0.77520
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 1.1488 - learning_rate: 1.0000e-04
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7324 - loss: 0.9678
Epoch 3: val_accuracy improved from 0.77520 to 0.84476, saving model to C:/Users/NAMITHAA/Downloads/save_models.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 702s 3s/step - accuracy: 0.7325 - loss: 0.9673 - val_accuracy: 0.8448 - val_loss: 0.5658 - learning_rate: 1.0000e-04
Epoch 4/10

Epoch 4: val_accuracy did not improve from 0.84476
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6875 - val_loss: 1.1054 - learning_rate: 1.0000e-04
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8231 - loss: 0.6285
Epoch 5: val_accuracy improved from 0.84476 to 0.88357, saving model to C:/Users/NAMITHAA/Downloads/save_models.keras
250/250 ━━━━━━━━━━━━━━━━━━